# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:
* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM Schema

In almost all cases, rather than just working with a single table you will typically need data from multiple tables. 
Doing this requires the use of **joins** using shared columns from the two tables. 

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [1]:
# Your code here
import sqlite3
import pandas as pd
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

## Display the names of all the employees in Boston 

Hint: join the employees and offices tables.

In [3]:
# Your code here
q = """SELECT lastName, firstName FROM employees JOIN offices USING (officeCode) WHERE city = 'Boston'"""
cur.execute(q).fetchall()

[('Firrelli', 'Julie'), ('Patterson', 'Steve')]

## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by.

In [12]:
# Your code here
q = """
    SELECT city, COUNT(officeCode) AS office_with_zero_employees
    FROM employees
    JOIN offices
    USING(officeCode)
    GROUP BY officeCode
    HAVING COUNT(employeeNumber) < 5
"""
cur.execute(q).fetchall()

[('Boston', 2), ('NYC', 2), ('Tokyo', 2), ('Sydney', 4), ('London', 2)]

## Write 3 Questions of your own and answer them

In [21]:
# Answers will vary
# Example: Display the htmlDescription and employee's first and last name for each product that each employee has sold
q = '''
    SELECT *
    FROM employees e
    JOIN customers c ON c.salesRepEmployeeNumber = e.employeeNumber
    JOIN orders o ON o.customerNumber = c.customerNumber
    JOIN orderDetails od ON od.orderNumber = o.orderNumber
    JOIN products p ON p.productCode = od.productCode
    JOIN productLines pl ON p.productLine = pl.productLine
    LIMIT 5
'''
cur.execute(q)
df = pd.DataFrame(cur.fetchall())
df.columns = [ i[0] for i in cur.description ]
df.loc[df['htmlDescription'] != '']

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle,customerNumber,customerName,...,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP,productLine,textDescription,htmlDescription,image


In [ ]:
# Your code here

In [ ]:
# Your code here

In [ ]:
# Your code here

## Level Up: Display the names of every individual product that each employee has sold

In pandas:

In [165]:
q = '''
    SELECT p.productName, e.firstName, e.lastName
    FROM products p
    JOIN orderDetails od ON p.productCode = od.productCode
    JOIN orders o ON o.orderNumber = od.orderNumber
    JOIN customers c ON c.customerNumber = o.customerNumber
    JOIN employees e ON e.employeeNumber = c.salesRepEmployeeNumber
'''
df = pd.DataFrame(cur.execute(q).fetchall())
df.columns = [ i[0] for i in cur.description ]
grouped = df.groupby(['firstName', 'lastName'])['productName'].value_counts() 
grouped

firstName  lastName   productName                        
Andy       Fixter     1913 Ford Model T Speedster            6
                      1940 Ford Pickup Truck                 5
                      1958 Setra Bus                         5
                      18th Century Vintage Horse Carriage    4
                      1928 Mercedes-Benz SSK                 4
                                                            ..
Steve      Patterson  Diamond T620 Semi-Skirted Tanker       1
                      HMS Bounty                             1
                      P-51-D Mustang                         1
                      Pont Yacht                             1
                      The Queen Mary                         1
Name: productName, Length: 1368, dtype: int64

In [169]:
q = '''
    SELECT p.productName, e.firstName, e.lastName
    FROM products p
    JOIN orderDetails od ON p.productCode = od.productCode
    JOIN orders o ON o.orderNumber = od.orderNumber
    JOIN customers c ON c.customerNumber = o.customerNumber
    JOIN employees e ON e.employeeNumber = c.salesRepEmployeeNumber
    GROUP BY e.firstName, e.lastName, p.productName
'''
df = pd.DataFrame(cur.execute(q).fetchall())
df.columns = [ i[0] for i in cur.description ]
df
# grouped = df.groupby(['firstName', 'lastName'])['productName'].value_counts() 
# grouped

,productName,firstName,lastName
0,18th Century Vintage Horse Carriage,Andy,Fixter
1,1900s Vintage Bi-Plane,Andy,Fixter
2,1900s Vintage Tri-Plane,Andy,Fixter
3,1911 Ford Town Car,Andy,Fixter
4,1913 Ford Model T Speedster,Andy,Fixter
...,...,...,...
1363,The Mayflower,Steve,Patterson
1364,The Queen Mary,Steve,Patterson
1365,The Schooner Bluenose,Steve,Patterson
1366,The Titanic,Steve,Patterson


## Level Up: Display the Number of Products each employee has sold

In [162]:
q = '''
    SELECT *
    FROM products p
    JOIN orderDetails od ON p.productCode = od.productCode
    JOIN orders o ON o.orderNumber = od.orderNumber
    JOIN customers c ON c.customerNumber = o.customerNumber
    JOIN employees e ON e.employeeNumber = c.salesRepEmployeeNumber
'''
df = pd.DataFrame(cur.execute(q).fetchall())
df.columns = [ i[0] for i in cur.description ]
df.groupby(['employeeNumber', 'firstName', 'lastName'])['productName'].count().sort_values(ascending=False)

employeeNumber  firstName  lastName 
1370            Gerard     Hernandez    396
1165            Leslie     Jennings     331
1401            Pamela     Castillo     272
1501            Larry      Bott         236
1504            Barry      Jones        220
1323            George     Vanauf       211
1612            Peter      Marsh        185
1611            Andy       Fixter       185
1337            Loui       Bondur       177
1216            Steve      Patterson    152
1286            Foon Yue   Tseng        142
1621            Mami       Nishi        137
1188            Julie      Firrelli     124
1702            Martin     Gerard       114
1166            Leslie     Thompson     114
Name: productName, dtype: int64

In SQL:

In [161]:
q = '''
    SELECT COUNT(productName) as product_count, e.firstName, e.lastName
    FROM products p
    JOIN orderDetails od ON p.productCode = od.productCode
    JOIN orders o ON o.orderNumber = od.orderNumber
    JOIN customers c ON c.customerNumber = o.customerNumber
    JOIN employees e ON e.employeeNumber = c.salesRepEmployeeNumber
    GROUP BY e.firstName, e.lastName
    ORDER BY product_count DESC
'''
df = pd.DataFrame(cur.execute(q).fetchall())
df.columns = [ i[0] for i in cur.description ]
df

,product_count,firstName,lastName
0,396,Gerard,Hernandez
1,331,Leslie,Jennings
2,272,Pamela,Castillo
3,236,Larry,Bott
4,220,Barry,Jones
5,211,George,Vanauf
6,185,Andy,Fixter
7,185,Peter,Marsh
8,177,Loui,Bondur
9,152,Steve,Patterson


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!